# Llm auto trainer

## Notebook Introduction: Fine-Tuning a Large Language Model with Ease

Welcome to this example notebook that demonstrates a simplified yet powerful approach to fine-tuning a Large Language Model (LLM) effortlessly. Fine-tuning is a crucial technique that allows you to adapt pre-trained language models to specific tasks, making them more contextually relevant and useful.

In this notebook, we will walk you through a step-by-step process of fine-tuning a state-of-the-art language model using a user-friendly and efficient method. You don't need to be an expert in machine learning or natural language processing to follow along – our approach focuses on simplicity and effectiveness.

### First, we will select the model we wish to fine-tune and take the matching tokenizer and appropriate config

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from transformers import logging

logging.set_verbosity("CRITICAL")

model_name = "tiiuae/falcon-7b"
tokenizer = model_name
generation_config = GenerationConfig.from_pretrained(model_name)

### Then, in order to use with mlrun, we will create an mlrun project and create an mlrun function

In [ ]:
import mlrun

project = mlrun.get_or_create_project(
    name="auto-trainer-test",
    context="./",
    user_project=True,
    parameters={
        "default_image": "yonishelach/mlrun-llm",
    },
)

In [ ]:
project.set_function(
    "auto-trainer.py",
    name="auto-trainer",
    kind="job",
    image="yonishelach/mlrun-llm",
    handler="finetune_llm",
)
project.save()

### we can set the every config or parameter we want, including training arguments, hyper parameters and more, and pass to the function

In [ ]:
import transformers

training_arguments = {
    "per_device_train_batch_size": 4,
    "gradient_accumulation_steps": 1,
    "warmup_steps": 2,
    "max_steps": 10,
    "learning_rate": 2e-4,
    "fp16": True,
    "logging_steps": 1,
    "optim": "paged_adamw_8bit",
}

### Now we simply run the function

In [ ]:
training_run = mlrun.run_function(
    function="auto-trainer",
    name="auto-trainer",
    local=True,
    params={
        "model": (model_name, "transformers.AutoModelForCausalLM"),
        "tokenizer": tokenizer,
        "train_dataset": "Abirate/english_quotes",
        "training_config": training_arguments,
        "quantization_config": True,
        "lora_config": True,
        "dataset_columns_to_train": "quote",
        "lora_target_modules": ["query_key_value"],
        "model_pretrained_config": {"trust_remote_code": True, "use_cache": False},
    },
    handler="finetune_llm",
    outputs=["model"],
)